# 데이터 수정(내 마음대로 수정)

In [ ]:
t = train_raw
t

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [ ]:
t["DDD"]=(t["DHI"] + t["DNI"])/2

In [ ]:
tt=t.drop('DHI',axis=1)
tt=tt.drop('DNI',axis=1)
tt

,Day,Hour,Minute,WS,RH,T,TARGET,DDD
0,0,0,0,1.5,69.08,-12,0.0,0.0
1,0,0,30,1.5,69.06,-12,0.0,0.0
2,0,1,0,1.6,71.78,-12,0.0,0.0
3,0,1,30,1.6,71.75,-12,0.0,0.0
4,0,2,0,1.6,75.20,-12,0.0,0.0
...,...,...,...,...,...,...,...,...
52555,1094,21,30,2.4,70.70,-4,0.0,0.0
52556,1094,22,0,2.4,66.79,-4,0.0,0.0
52557,1094,22,30,2.2,66.78,-4,0.0,0.0
52558,1094,23,0,2.1,67.72,-4,0.0,0.0


In [ ]:
dd=(t["RH"] + t["DHI"])
dd

0        69.08
1        69.06
2        71.78
3        71.75
4        75.20
         ...  
52555    70.70
52556    66.79
52557    66.78
52558    67.72
52559    67.70
Length: 52560, dtype: float64

0        34.540
1        34.530
2        35.890
3        35.875
4        37.600
          ...  
52555    35.350
52556    33.395
52557    33.390
52558    33.860
52559    33.850
Length: 52560, dtype: float64

In [ ]:
def create_lag_feats(data, lags, cols):                                         # 여기서 lag_48을 넣어주고 하는건데 왜 하는거지?
    
    lag_cols = []
    temp = data.copy()
    for col in cols:
        for lag in lags:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            temp['Target1'] = temp['TARGET']  #
            temp['Target2'] = temp['TARGET'].shift(-48).fillna(method='ffill')  # target2에는 target에 -48 위치의 행값으로 시작하고 결측치는 앞의 행값으로 채움 
            lag_cols.append(col + '_lag_%s'%lag)                                # 왜 48을 빼고 target2로 넣는거지? 48은 하루이다. 30분씩 측정되어있으니 48이 하루

    return temp, lag_cols
    

def preprocess_data(data, target_lags=[48], weather_lags=[48], is_train=True):
    
    temp = data.copy()

    if is_train==True:          
    
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DDD', 'WS', 'RH', 'T'])
     
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2 + ['Target1', 'Target2']].dropna()  # temp의 Hour을 가져오고 거기에 temp_lag_칼럼들 1,2 가져옴
                                                                                                   # 이건 train이라 target1이랑 2도 합치는것.
    elif is_train==False:    
        
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DDD', 'WS', 'RH', 'T'])
                              
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2].dropna()


df_train = preprocess_data(tt, target_lags=[48], weather_lags=[48], is_train=True)
df_train.iloc[:48]

,Hour,TARGET_lag_48,DDD_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.000000,0.0,1.5,69.08,-12.0,0.000000,0.000000
49,0,0.000000,0.0,1.5,69.06,-12.0,0.000000,0.000000
50,1,0.000000,0.0,1.6,71.78,-12.0,0.000000,0.000000
51,1,0.000000,0.0,1.6,71.75,-12.0,0.000000,0.000000
52,2,0.000000,0.0,1.6,75.20,-12.0,0.000000,0.000000
53,2,0.000000,0.0,1.5,69.29,-11.0,0.000000,0.000000
54,3,0.000000,0.0,1.5,72.56,-11.0,0.000000,0.000000
55,3,0.000000,0.0,1.4,72.55,-11.0,0.000000,0.000000
56,4,0.000000,0.0,1.3,74.62,-11.0,0.000000,0.000000
57,4,0.000000,0.0,1.3,74.61,-11.0,0.000000,0.000000


In [ ]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, target_lags=[48], weather_lags=[48], is_train=False).iloc[-48:]
    df_test.append(temp)

X_test = pd.concat(df_test)
X_test.shape

# LGBM 수정 (XXX 안봐도 됨)

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_raw = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/train/train.csv')
train= train_raw

In [ ]:
train.tail()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0
52559,1094,23,30,0,0,2.1,67.70,-4,0.0


In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/sample_submission.csv')

In [ ]:
submission.tail()

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7775,80.csv_Day8_23h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def create_lag_feats(data, lags, cols):                                         # 여기서 lag_48을 넣어주고 하는건데 왜 하는거지?
    
    lag_cols = []
    temp = data.copy()
    for col in cols:
        for lag in lags:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            temp['Target1'] = temp['TARGET']  #
            temp['Target2'] = temp['TARGET'].shift(-48).fillna(method='ffill')  # target2에는 target에 -48 위치의 행값으로 시작하고 결측치는 앞의 행값으로 채움 
            lag_cols.append(col + '_lag_%s'%lag)                                # 왜 48을 빼고 target2로 넣는거지? 48은 하루이다. 30분씩 측정되어있으니 48이 하루
                                                                                # 전날꺼를 빼고 다음날꺼를 보는건가?

    return temp, lag_cols
    

def preprocess_data(data, target_lags=[48], weather_lags=[48], is_train=True):
    
    temp = data.copy()

    if is_train==True:          
    
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
     
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2 + ['Target1', 'Target2']].dropna()  # temp의 Hour을 가져오고 거기에 temp_lag_칼럼들 1,2 가져옴
                                                                                                   # 이건 train이라 target1이랑 2도 합치는것.
    elif is_train==False:    
        
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
                              
        return temp[['Hour'] + temp_lag_cols1 + temp_lag_cols2].dropna()


df_train = preprocess_data(train, target_lags=[48], weather_lags=[48], is_train=True)
df_train.iloc[:48]

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.000000,0.0,0.0,1.5,69.08,-12.0,0.000000,0.000000
49,0,0.000000,0.0,0.0,1.5,69.06,-12.0,0.000000,0.000000
50,1,0.000000,0.0,0.0,1.6,71.78,-12.0,0.000000,0.000000
51,1,0.000000,0.0,0.0,1.6,71.75,-12.0,0.000000,0.000000
52,2,0.000000,0.0,0.0,1.6,75.20,-12.0,0.000000,0.000000
53,2,0.000000,0.0,0.0,1.5,69.29,-11.0,0.000000,0.000000
54,3,0.000000,0.0,0.0,1.5,72.56,-11.0,0.000000,0.000000
55,3,0.000000,0.0,0.0,1.4,72.55,-11.0,0.000000,0.000000
56,4,0.000000,0.0,0.0,1.3,74.62,-11.0,0.000000,0.000000
57,4,0.000000,0.0,0.0,1.3,74.61,-11.0,0.000000,0.000000


In [ ]:
df_train

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.0,0.0,0.0,1.5,69.08,-12.0,0.0,0.0
49,0,0.0,0.0,0.0,1.5,69.06,-12.0,0.0,0.0
50,1,0.0,0.0,0.0,1.6,71.78,-12.0,0.0,0.0
51,1,0.0,0.0,0.0,1.6,71.75,-12.0,0.0,0.0
52,2,0.0,0.0,0.0,1.6,75.20,-12.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
52555,21,0.0,0.0,0.0,2.4,68.38,-2.0,0.0,0.0
52556,22,0.0,0.0,0.0,2.4,71.09,-3.0,0.0,0.0
52557,22,0.0,0.0,0.0,2.2,71.11,-3.0,0.0,0.0
52558,23,0.0,0.0,0.0,2.1,74.99,-4.0,0.0,0.0


In [ ]:
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [ ]:
train.iloc[48:96]

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
48,1,0,0,0,0,1.6,90.66,-10,0.000000
49,1,0,30,0,0,1.6,90.68,-10,0.000000
50,1,1,0,0,0,1.6,88.11,-11,0.000000
51,1,1,30,0,0,1.6,88.11,-11,0.000000
52,1,2,0,0,0,1.6,90.85,-11,0.000000
53,1,2,30,0,0,1.6,90.84,-11,0.000000
54,1,3,0,0,0,1.7,93.78,-12,0.000000
55,1,3,30,0,0,1.7,93.77,-12,0.000000
56,1,4,0,0,0,1.7,90.46,-12,0.000000
57,1,4,30,0,0,1.6,90.46,-12,0.000000


In [ ]:
df_train.tail()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
52555,21,0.0,0.0,0.0,2.4,68.38,-2.0,0.0,0.0
52556,22,0.0,0.0,0.0,2.4,71.09,-3.0,0.0,0.0
52557,22,0.0,0.0,0.0,2.2,71.11,-3.0,0.0,0.0
52558,23,0.0,0.0,0.0,2.1,74.99,-4.0,0.0,0.0
52559,23,0.0,0.0,0.0,1.9,74.99,-4.0,0.0,0.0


In [ ]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, target_lags=[48], weather_lags=[48], is_train=False).iloc[-48:]
    df_test.append(temp)

X_test = pd.concat(df_test)
X_test.shape

(23328, 7)

In [ ]:
X_test.head()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48
48,0,0.0,0.0,0.0,2.7,34.42,0.0
49,0,0.0,0.0,0.0,2.7,34.17,0.1
50,1,0.0,0.0,0.0,2.7,34.23,0.2
51,1,0.0,0.0,0.0,2.7,33.99,0.3
52,2,0.0,0.0,0.0,2.8,33.97,0.4


In [ ]:
X_test

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48
48,0,0.0,0.0,0.0,2.7,34.42,0.0
49,0,0.0,0.0,0.0,2.7,34.17,0.1
50,1,0.0,0.0,0.0,2.7,34.23,0.2
51,1,0.0,0.0,0.0,2.7,33.99,0.3
52,2,0.0,0.0,0.0,2.8,33.97,0.4
...,...,...,...,...,...,...,...
331,21,0.0,0.0,0.0,0.5,74.13,12.5
332,22,0.0,0.0,0.0,0.7,73.54,12.0
333,22,0.0,0.0,0.0,0.9,75.01,11.7
334,23,0.0,0.0,0.0,1.1,74.47,11.3


In [ ]:
X_test.duplicated().sum()

45

In [ ]:
X_test[X_test.duplicated()]

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48
331,21,0.0,0.0,0.0,0.5,82.74,-2.6
329,20,0.0,0.0,0.0,1.7,58.56,-11.4
331,21,0.0,0.0,0.0,1.8,58.17,-11.4
333,22,0.0,0.0,0.0,1.8,57.15,-11.3
335,23,0.0,0.0,0.0,3.3,62.46,-1.4
333,22,0.0,0.0,0.0,2.1,54.12,-4.3
327,19,0.0,0.0,0.0,2.0,73.02,-2.4
333,22,0.0,0.0,0.0,1.8,73.50,-2.5
289,0,0.0,0.0,0.0,1.0,72.94,1.5
291,1,0.0,0.0,0.0,1.0,71.24,1.6


In [ ]:
df_train.head()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48,Target1,Target2
48,0,0.0,0.0,0.0,1.5,69.08,-12.0,0.0,0.0
49,0,0.0,0.0,0.0,1.5,69.06,-12.0,0.0,0.0
50,1,0.0,0.0,0.0,1.6,71.78,-12.0,0.0,0.0
51,1,0.0,0.0,0.0,1.6,71.75,-12.0,0.0,0.0
52,2,0.0,0.0,0.0,1.6,75.20,-12.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.3, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.3, random_state=0)

In [ ]:
X_train_1.head()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48
22935,19,0.0,0.0,0.0,2.0,60.54,11.0
8835,1,0.0,0.0,0.0,0.7,100.00,15.0
16307,17,0.0,0.0,0.0,1.5,59.30,-2.0
29195,5,0.0,0.0,0.0,1.5,97.35,12.0
4459,21,0.0,0.0,0.0,3.2,76.76,-2.0


In [ ]:
X_test.head()

,Hour,TARGET_lag_48,DHI_lag_48,DNI_lag_48,WS_lag_48,RH_lag_48,T_lag_48
52512,0,0.0,0.0,0.0,4.1,55.00,-2.0
52513,0,0.0,0.0,0.0,3.9,54.99,-2.0
52514,1,0.0,0.0,0.0,3.7,54.34,-3.0
52515,1,0.0,0.0,0.0,3.3,54.34,-2.0
52516,2,0.0,0.0,0.0,3.0,54.73,-2.0


In [ ]:
quantiles = [0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95]

In [ ]:
from lightgbm import LGBMRegressor

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)                   
                         
                         
    model.fit(X_train, Y_train, eval_metric = ['quantile'], 
          eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model and the predictions in (a) - (b)
def GBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
   # (a) Modeling  
   mod = GradientBoostingRegressor(loss='quantile', alpha=q,
                                n_estimators=500, max_depth=8,
                                learning_rate=.01, min_samples_leaf=20,
                                min_samples_split=20)
   mod.fit(X_train_1, Y_train_1)

   # (b) Predictions
   pred = pd.Series(mod.predict(X_test).round(2))
   return pred, mod

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

  GBM_models=[]
  GBM_actual_pred = pd.DataFrame()

  for q in quantiles:
      pred , model = GBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
      GBM_models.append(model)
      GBM_actual_pred = pd.concat([GBM_actual_pred,pred],axis=1)
  GBM_actual_pred.columns = quantiles

  return GBM_models, GBM_actual_pred


In [ ]:
def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred , model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [ ]:
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
results_2.sort_index()[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
results_1.sort_index().iloc[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
results_2.sort_index()

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
1,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
2,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
4,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3884,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3885,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3886,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4


In [ ]:
print(results_1.shape, results_2.shape)

(3888, 9) (3888, 9)


In [ ]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4


In [ ]:
submission.iloc[:48]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.csv_Day7_0h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.csv_Day7_1h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.csv_Day7_1h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.csv_Day7_2h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.csv_Day7_2h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.csv_Day7_3h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.csv_Day7_3h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.csv_Day7_4h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.csv_Day7_4h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
submission.iloc[48:96]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
48,0.csv_Day8_0h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
49,0.csv_Day8_0h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
50,0.csv_Day8_1h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
51,0.csv_Day8_1h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
52,0.csv_Day8_2h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
53,0.csv_Day8_2h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
54,0.csv_Day8_3h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
55,0.csv_Day8_3h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
56,0.csv_Day8_4h00m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
57,0.csv_Day8_4h30m,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
submission.to_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/submission.csv', index=False)

# GBM


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model and the predictions in (a) - (b)
def GBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
   # (a) Modeling  
   mod = GradientBoostingRegressor(loss='quantile', alpha=q,
                                n_estimators=500, max_depth=8,
                                learning_rate=.01, min_samples_leaf=20,
                                min_samples_split=20)
   mod.fit(X_train_1, Y_train_1)

   # (b) Predictions
   pred = pd.Series(mod.predict(X_test).round(2))
   return pred, mod

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

  GBM_models=[]
  GBM_actual_pred = pd.DataFrame()

  for q in quantiles:
      pred , model = GBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
      GBM_models.append(model)
      GBM_actual_pred = pd.concat([GBM_actual_pred,pred],axis=1)
  GBM_actual_pred.columns = quantiles

  return GBM_models, GBM_actual_pred


## 수정된 baseline


In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_raw = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/train/train.csv')
train= train_raw

In [4]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/sample_submission.csv')

In [5]:
train.iloc[52511:]

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
52511,1093,23,30,0,0,1.9,74.99,-4,0.000000
52512,1094,0,0,0,0,1.7,74.22,-4,0.000000
52513,1094,0,30,0,0,1.6,74.23,-4,0.000000
52514,1094,1,0,0,0,1.5,74.64,-4,0.000000
52515,1094,1,30,0,0,1.7,74.65,-4,0.000000
52516,1094,2,0,0,0,2.0,74.35,-4,0.000000
52517,1094,2,30,0,0,2.3,74.35,-4,0.000000
52518,1094,3,0,0,0,2.7,72.25,-4,0.000000
52519,1094,3,30,0,0,2.7,72.26,-4,0.000000
52520,1094,4,0,0,0,2.8,70.20,-4,0.000000


In [6]:
train.iloc[:48]

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.000000
1,0,0,30,0,0,1.5,69.06,-12,0.000000
2,0,1,0,0,0,1.6,71.78,-12,0.000000
3,0,1,30,0,0,1.6,71.75,-12,0.000000
4,0,2,0,0,0,1.6,75.20,-12,0.000000
5,0,2,30,0,0,1.5,69.29,-11,0.000000
6,0,3,0,0,0,1.5,72.56,-11,0.000000
7,0,3,30,0,0,1.4,72.55,-11,0.000000
8,0,4,0,0,0,1.3,74.62,-11,0.000000
9,0,4,30,0,0,1.3,74.61,-11,0.000000


In [7]:
train['Target1'] = train['TARGET'].shift(-48).fillna(method='ffill')
train.iloc[49:96]

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Target1
49,1,0,30,0,0,1.6,90.68,-10,0.000000,0.000000
50,1,1,0,0,0,1.6,88.11,-11,0.000000,0.000000
51,1,1,30,0,0,1.6,88.11,-11,0.000000,0.000000
52,1,2,0,0,0,1.6,90.85,-11,0.000000,0.000000
53,1,2,30,0,0,1.6,90.84,-11,0.000000,0.000000
54,1,3,0,0,0,1.7,93.78,-12,0.000000,0.000000
55,1,3,30,0,0,1.7,93.77,-12,0.000000,0.000000
56,1,4,0,0,0,1.7,90.46,-12,0.000000,0.000000
57,1,4,30,0,0,1.6,90.46,-12,0.000000,0.000000
58,1,5,0,0,0,1.6,87.34,-12,0.000000,0.000000


In [ ]:
xx=train_raw

#xx.iloc[:-96]
xx

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [8]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96] # 왜 마지막 2일을 빼고 학습?

    elif is_train==False:
       
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :] # 왜 마지막 날들만 빼서 데이터를 모으지?


df_train = preprocess_data(train)
df_train.iloc[:48]

,Hour,TARGET,DHI,DNI,WS,RH,T,Target1,Target2
0,0,0.000000,0,0,1.5,69.08,-12,0.000000,0.000000
1,0,0.000000,0,0,1.5,69.06,-12,0.000000,0.000000
2,1,0.000000,0,0,1.6,71.78,-12,0.000000,0.000000
3,1,0.000000,0,0,1.6,71.75,-12,0.000000,0.000000
4,2,0.000000,0,0,1.6,75.20,-12,0.000000,0.000000
5,2,0.000000,0,0,1.5,69.29,-11,0.000000,0.000000
6,3,0.000000,0,0,1.5,72.56,-11,0.000000,0.000000
7,3,0.000000,0,0,1.4,72.55,-11,0.000000,0.000000
8,4,0.000000,0,0,1.3,74.62,-11,0.000000,0.000000
9,4,0.000000,0,0,1.3,74.61,-11,0.000000,0.000000


In [9]:
df_train.tail()

,Hour,TARGET,DHI,DNI,WS,RH,T,Target1,Target2
52459,21,0.0,0,0,3.5,55.97,-1,0.0,0.0
52460,22,0.0,0,0,3.9,54.23,-2,0.0,0.0
52461,22,0.0,0,0,4.1,54.21,-2,0.0,0.0
52462,23,0.0,0,0,4.3,56.46,-2,0.0,0.0
52463,23,0.0,0,0,4.1,56.44,-2,0.0,0.0


In [38]:
df_test = []

for i in range(81):
    file_path = '/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

X_test = pd.concat(df_test)
X_test

## 이건 preprocess_data라는 함수때문에 각각 test 파일의 마지막 하루만 뽑아서 81일동안 합쳐서 3888개의 행이 나온거임
## 왜 마지막 날만 뽑았을까?

,Hour,TARGET,DHI,DNI,WS,RH,T
288,0,0.0,0,0,0.8,80.92,-2.8
289,0,0.0,0,0,0.9,81.53,-2.9
290,1,0.0,0,0,1.0,79.91,-3.0
291,1,0.0,0,0,0.9,79.91,-3.0
292,2,0.0,0,0,0.9,77.20,-3.0
...,...,...,...,...,...,...,...
331,21,0.0,0,0,0.8,63.35,13.7
332,22,0.0,0,0,0.7,64.82,13.1
333,22,0.0,0,0,0.7,66.10,12.8
334,23,0.0,0,0,0.6,67.64,12.4


In [36]:
X_test = X_test[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
X_test

,Hour,TARGET,DHI,DNI,WS,RH,T
288,0,0.0,0,0,0.8,80.92,-2.8
289,0,0.0,0,0,0.9,81.53,-2.9
290,1,0.0,0,0,1.0,79.91,-3.0
291,1,0.0,0,0,0.9,79.91,-3.0
292,2,0.0,0,0,0.9,77.20,-3.0
...,...,...,...,...,...,...,...
331,21,0.0,0,0,0.8,63.35,13.7
332,22,0.0,0,0,0.7,64.82,13.1
333,22,0.0,0,0,0.7,66.10,12.8
334,23,0.0,0,0,0.6,67.64,12.4


In [12]:
df_train.iloc[:, -1]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
52459    0.0
52460    0.0
52461    0.0
52462    0.0
52463    0.0
Name: Target2, Length: 52464, dtype: float64

In [13]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.3, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.3, random_state=0)

In [ ]:
#X_train_1.head()
X_valid_1.head()

,Hour,TARGET,DHI,DNI,WS,RH,T
47237,2,0.000000,0,0,1.1,80.65,14
49429,18,0.000000,0,0,0.5,53.20,0
27337,12,84.996275,330,615,3.4,23.19,32
28311,19,0.000000,0,0,0.7,50.55,17
5361,16,21.957266,185,116,0.6,45.08,13


In [ ]:
# 이거 밑에는 lgbm

In [ ]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
from lightgbm import LGBMRegressor

# Get the model and the predictions in (a) - (b)
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) Modeling  
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)                   
                         
                         
    model.fit(X_train, Y_train, eval_metric = ['quantile'], 
          eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)

    # (b) Predictions
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [ ]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred , model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

## GBM

In [ ]:
#데이터 전처리한것은 앞에 수정된 Baseline꺼 그대로 사용하고 모델만 GBM으로 바꿔서 적용한것
#이게 현재까지는 젤 점수 높게 나왔음

# 데이콘에 baseline code는 LGBM이고 이건 돌리는데 조금 오래걸리지만 해본거임

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model and the predictions in (a) - (b)
def GBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
   # (a) Modeling  
   mod = GradientBoostingRegressor(loss='quantile', alpha=q,
                                n_estimators=500, max_depth=8,
                                learning_rate=.01, min_samples_leaf=20,
                                min_samples_split=20)
   mod.fit(X_train, Y_train)

   # (b) Predictions
   pred = pd.Series(mod.predict(X_test).round(2))
   return pred, mod

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

  GBM_models=[]
  GBM_actual_pred = pd.DataFrame()

  for q in quantiles:
      pred , model = GBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
      GBM_models.append(model)
      GBM_actual_pred = pd.concat([GBM_actual_pred,pred],axis=1)
  GBM_actual_pred.columns = quantiles

  return GBM_models, GBM_actual_pred

In [ ]:
# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
# Target2
models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
results_2.sort_index()[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
results_1.sort_index().iloc[:48]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
1,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
2,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
3,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
4,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
5,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
6,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
7,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
8,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40
9,0.00,0.00,0.00,0.00,0.00,0.06,0.15,0.27,0.40


In [ ]:
results_2.sort_index()

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
1,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
2,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
3,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
4,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
...,...,...,...,...,...,...,...,...,...
27211,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
27212,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
27213,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4
27214,0.0,0.0,0.0,0.0,0.0,0.06,0.15,0.27,0.4


In [ ]:
print(results_1.shape, results_2.shape)

(27216, 9) (27216, 9)


In [ ]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission

ValueError: ignored

In [ ]:
submission.to_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/GBMsub.csv', index=False)

# 랜덤 포레스트 (미완성)

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_raw = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/train/train.csv')
train= train_raw

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/sample_submission.csv')

In [ ]:
def transform(dataset, target, start_index, end_index, history_size,
                      target_size, step):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index, 48):
        indices = range(i-history_size, i, step)
        data.append(np.ravel(dataset[indices].T))
        labels.append(target[i:i+target_size])
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

# x_col =['DHI','DNI','WS','RH','T','TARGET']
x_col =['TARGET']
y_col = ['TARGET']

dataset = train.loc[:,x_col].values
label = np.ravel(train.loc[:,y_col].values)

past_history = 48 * 2
future_target = 48 * 2

### transform train
train_data, train_label = transform(dataset, label, 0,None, past_history,future_target, 1)
### transform test
test = []
for i in range(81):
    data = []
    tmp = pd.read_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/test/' + str(i) + '.csv')
    tmp = tmp.loc[:, x_col].values
    tmp = tmp[-past_history:,:]
    data.append(np.ravel(tmp.T))
    data = np.array(data)
    test.append(data)
test = np.concatenate(test, axis=0)

In [ ]:
from sklearn import ensemble
N_ESTIMATORS = 1000
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS,
                                    max_features=1, random_state=0,
                                    max_depth = 5,
                                    verbose=True,
                                    n_jobs=-1)
rf.fit(train_data, train_label)

rf_preds = []
for estimator in rf.estimators_:
    rf_preds.append(estimator.predict(test))
rf_preds = np.array(rf_preds)

for i, q in enumerate(np.arange(0.1, 1, 0.1)):
    y_pred = np.percentile(rf_preds, q * 100, axis=0)
    submission.iloc[:, i] = np.ravel(y_pred)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.3s finished


In [ ]:
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
submission.to_csv('/content/drive/MyDrive/Dacon/태양광 발전량 예측 AI 경진대회/랜포1.csv', index=True)

#랜포(깃허브)

In [14]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.regression.quantile_regression as Q_reg
import matplotlib.pyplot as plt

In [34]:
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -2], test_size=0.3, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:, :-2], df_train.iloc[:, -1], test_size=0.3, random_state=0)

In [39]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model, the cofficients and the predictions in (a) - (c)
def Qreg(q, X_train, Y_train, X_valid, Y_valid, X_test):
   # (a) Modeling 
   mod = Q_reg.QuantReg(Y_train, X_train).fit(q=q)

    
   # (c) Predictions
   pred = pd.Series(mod.predict(X_test).round(2))
   return pred, mod

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):
  
  RF_models= []
  Qreg_actual_pred = pd.DataFrame()

# Get the actual and quantile predictions
  for q in quantiles:
      pred, model = Qreg(X_train, Y_train, X_valid, Y_valid, X_test)
      RF_models.append(model)
      Qreg_actual_pred = pd.concat([Qreg_actual_pred,pred],axis=1)

  Qreg_actual_pred.columns=quantiles
  Qreg_actual_pred

In [43]:
X_test

,Hour,TARGET,DHI,DNI,WS,RH,T
288,0,0.0,0,0,0.8,80.92,-2.8
289,0,0.0,0,0,0.9,81.53,-2.9
290,1,0.0,0,0,1.0,79.91,-3.0
291,1,0.0,0,0,0.9,79.91,-3.0
292,2,0.0,0,0,0.9,77.20,-3.0
...,...,...,...,...,...,...,...
331,21,0.0,0,0,0.8,63.35,13.7
332,22,0.0,0,0,0.7,64.82,13.1
333,22,0.0,0,0,0.7,66.10,12.8
334,23,0.0,0,0,0.6,67.64,12.4


In [44]:
# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

TypeError: ignored

In [29]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Get the model, the cofficients and the predictions in (a) - (c)
def Qreg(q):
   # (a) Modeling 
   mod2 = Q_reg.QuantReg(Y_train_2, X_train_2).fit(q=q)

    
   # (c) Predictions
   pred = pd.Series(mod2.predict(X_test).round(2))
   return pred


Qreg_actual_pred = pd.DataFrame()

# Get the actual and quantile predictions
for q in quantiles:
    pred = Qreg(q)
    Qreg_actual_pred2 = pd.concat([Qreg_actual_pred,pred],axis=1)

Qreg_actual_pred2.columns=quantiles
Qreg_actual_pred2

ValueError: ignored

In [15]:
# random forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, random_state=0,min_samples_split=10)
rf.fit(X_train_1, Y_train_1)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
# Get the predictions of all trees for all observations
# Each observation has N predictions from the N trees
pred_Q = pd.DataFrame()
for pred in rf.estimators_:
    temp = pd.Series(pred.predict(X_test).round(2))
    pred_Q = pd.concat([pred_Q,temp],axis=1)
pred_Q.head()

In [ ]:
RF_actual_pred = pd.DataFrame()

for q in quantiles:
    s = pred_Q.quantile(q=q, axis=1)
    RF_actual_pred = pd.concat([RF_actual_pred,s],axis=1,sort=False)
   
RF_actual_pred.columns=quantiles
RF_actual_pred['actual'] = Y_test
RF_actual_pred['interval'] = RF_actual_pred[np.max(quantiles)] - RF_actual_pred[np.min(quantiles)]
RF_actual_pred = RF_actual_pred.sort_values('interval')
RF_actual_pred = RF_actual_pred.round(2)
RF_actual_pred